In [3]:
import matplotlib.pyplot as plt
import numpy as np
# Load MNIST dataset
train_images=np.load('/content/drive/MyDrive/datasets/mnist.npz (Unzipped Files)/x_train.npy')
train_labels=np.load('/content/drive/MyDrive/datasets/mnist.npz (Unzipped Files)/y_train.npy')
test_images=np.load('/content/drive/MyDrive/datasets/mnist.npz (Unzipped Files)/x_test.npy')
test_labels=np.load('/content/drive/MyDrive/datasets/mnist.npz (Unzipped Files)/y_test.npy')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data=[]
print(train_images.shape)
lab=[]

for i in range(3):
  data.extend(train_images[train_labels == i])
  lab.extend(train_labels[train_labels==i])
lab=np.array(lab)
print
data=np.array(data)
data=data.reshape(-1,784)
N=data.shape[0]
data=data.T
print(data.shape)
def pca(data,N):
  mean=np.mean(data,axis=1, keepdims=True)

  print(mean.shape)
  data=data-(mean)
  cov=(1/(N-1))*np.dot(data,data.T)

  eg_value,eg_vector=np.linalg.eigh(cov)

  eg=[]
  eg_vector=eg_vector.T
  for i in range(len(eg_value)):
    eg.append((eg_value[i],eg_vector[i]))

  sorted_keys=sorted(eg, key=lambda x: x[0])[::-1]
  sk2=sorted_keys[:10]
  # print([i[0] for i in sk2])
  # print(sorted_keys[0][0])
  U=[]
  # print(len(sorted_keys))
  for i,j in sorted_keys:
    U.append(j)
  U=np.array(U)
  Up=U[:10]
  y=np.dot(Up,data)
  return Up,mean,data
Up,mean,data=pca(data,N)
y=np.dot(Up,data)
orig_y=y
print(y.shape)
yT=y.T
regions=[]
for i in range(10):
  regions.append([min(y[i]),max(y[i])])

print(regions)
boundaries=[]
def validPointForTrain(obj,regions):
  for i in range(10):
    # print(obj[i],regions[i][0],regions[i][1])
    if obj[i]<regions[i][0] or obj[i] >= regions[i][1]:
      return False
  return True
def validPointForTest(obj,regions):

  for i in regions:
    # print(obj[i],regions[i][0],regions[i][1])
    #(dim,mid,l/r)
    if (i[2]=='l' and obj[i[0]] > i[1]):
      return False
    if (i[2]=='r' and obj[i[0]] < i[1]):
      return False
  return True
intvls=[]
def learn_tree(y,yT,data,nodes,lab,dim,regions,decisions,boundaries):

  min_g=()
  for j in range(dim):
    d=y[j]
    sorted_d=sorted(d)
    me=0
    cnt=0
    x=(regions[j][0]+regions[j][1])/2

    # print(regions[j][0],regions[j][1],x)

    p=[0,0,0]
    q=[0,0,0]
    for k in range(len(lab)):
      if not validPointForTest(yT[k],boundaries):
        continue
        # print(k)
        # return
      if yT[k][j]<x:
        p[lab[k]]+=1
      else:
        q[lab[k]]+=1
    # print("lengths:",len(p),len(q))
    # print(p,x)
    print(p,q)
    ps=sum(p)
    qs=sum(q)
    # print(p,q)
    gp=0
    gq=0
    # print(len(p),len(q),ps,qs)
    for k in range(3):
      p[k]/=1 if ps==0 else ps
      # print(ps,qs,k)
      q[k]/=1 if qs==0 else qs
      gp+=p[k]*(1-p[k])
      gq+=q[k]*(1-q[k])
    g=(gp*ps+gq*qs)/(ps+qs)
    pi=0
    pv=p[0]
    qi=0
    qv=q[0]
    for k in range(1,3):
      if p[k]>pv:
        pv=p[k]
        pi=k
      if q[k]>qv:
        qv=q[k]
        qi=k
    if j==0 or g<min_g[0]:
      min_g=(g,j,gp,gq,pi,qi,x)
  b1=[i for i in boundaries]
  b1.append((min_g[1],min_g[-1],'l'))
  # print("min_g",min_g,nodes)
  b2=[i for i in boundaries]
  b2.append((min_g[1],min_g[-1]+1,'r'))

  r1=[[i,j] for i,j in regions]
  r2=[[i,j] for i,j in regions]
  r1[min_g[1]][1]=min_g[-1]
  r2[min_g[1]][0]=min_g[-1]+1
  if(nodes==2):

    decisions.append((b2,min_g[5]))
    decisions.append((b1,min_g[4]))

    return
  # return
  # print(min_g)
  if(min_g[2]<min_g[3]):
    decisions.append((b1,min_g[4]))

    learn_tree(y,yT,data,nodes-1,lab,10,r2,decisions,b2)
  else:
    decisions.append((b2,min_g[5]))
    learn_tree(y,yT,data,nodes-1,lab,10,r1,decisions,b1)

decisions=[]
boundaries=[]
learn_tree(y,yT,data,3,lab,10,regions,decisions,boundaries)
print(decisions)
# print([i[1] for i in decisions])
totals=[0,0,0]
corrects=[0,0,0]
def find_class(decisions,elem):
  ans=-1
  for i in decisions:
    if validPointForTest(elem,i[0]):
      ans=i[1]
  if(ans==-1):
    print(ans)
  return ans
orig_lab=lab
test=[]
test_labls=[]
for i in range(3):
  test.extend(test_images[test_labels == i])
  test_labls.extend(test_labels[test_labels==i])
test=np.array(test).reshape(-1,784).T-mean
test_labls=np.array(test_labls)
N=data.shape[1]
# print(N)
# print(Up.shape)
test_y=np.dot(Up,test).T
# print(test_y.shape,test_labls.shape)

for i in range(len(test_labls)):


  if test_labls[i]==find_class(decisions,test_y[i]):
    corrects[test_labls[i]]+=1
  totals[test_labls[i]]+=1
for i in range(3):
  print(corrects[i]/totals[i])
print(sum(corrects)/sum(totals))
















(60000, 28, 28)
(784, 18623)
(784, 1)
(10, 18623)
[[-1153.0032059549321, 2309.469659023523], [-1744.7925719253808, 1143.1769568995567], [-1587.730798890112, 1558.6794306807906], [-1136.7091743694577, 1078.1487102903764], [-1425.9842536377487, 1163.3148133319085], [-1116.1022112093983, 1441.3195305277177], [-1204.9315328970388, 1281.6954743537297], [-873.5818077308855, 1235.419083962722], [-1352.2570434450204, 1257.8336071821027], [-1116.99226621869, 1030.5170803574915]]
[979, 6742, 5494] [4944, 0, 464]
[301, 715, 4610] [5622, 6027, 1348]
[2864, 1582, 3577] [3059, 5160, 2381]
[2108, 3243, 4148] [3815, 3499, 1810]
[2663, 2332, 1627] [3260, 4410, 4331]
[3944, 5020, 4023] [1979, 1722, 1935]
[2871, 4435, 3509] [3052, 2307, 2449]
[4841, 4410, 4358] [1082, 2332, 1600]
[2488, 2340, 2696] [3435, 4402, 3262]
[2501, 3175, 2832] [3422, 3567, 3126]
[18, 6697, 1675] [961, 45, 3819]
[114, 715, 4209] [865, 6027, 1285]
[424, 1582, 3428] [555, 5160, 2066]
[544, 3243, 3715] [435, 3499, 1779]
[532, 2332, 

In [5]:
import random
max_acc=[0,[]]

for w in range(1):
  all_data=[]
  all_lab=[]
  for i in range(3):
    all_data.extend(train_images[train_labels == i])
    all_lab.extend(train_labels[train_labels==i])
  d5=[[],[],[],[],[]]
  l5=[[],[],[],[],[]]
  # for i in range(len(all_lab)):
  #   for j in range(5):
  #     idx=random.randint(0,len(all_lab)-1)
  #     d5[j].append(all_data[idx]);
  #     l5[j].append(all_lab[idx]);
  ids=[i for  i in range(len(all_lab))]
  for i in range(5):
    idxs=random.choices(ids,k=len(all_lab))
    d5[i]=[all_data[j] for j in idxs]
    l5[i]=[all_lab[j] for j in idxs]
  print(len(d5[0]),len(d5[1]),len(d5[2]),len(d5[3]),len(d5[4]))

  trees=[]
  for j in range(5):
    d=d5[j]
    d=np.array(d)
    d=d.reshape(-1,784)
    N=d.shape[0]
    d=d.T
    up,mean,data=pca(d,N)
    yp=np.dot(up,data)
    regions=[]
    for i in range(10):
      regions.append([min(yp[i]),max(yp[i])])

    tree=[]

    learn_tree(yp,yp.T,d,3,l5[j],10,regions,tree,[])
    trees.append(tree)
  test=[]
  test_labls=[]
  for i in range(3):
    test.extend(test_images[test_labels == i])
    test_labls.extend(test_labels[test_labels==i])
  test=np.array(test).reshape(-1,784).T-mean
  test_labls=np.array(test_labls)
  N=data.shape[1]
  print(N)
  print(Up.shape)
  test_y=np.dot(Up,test).T
  print(test_y.shape,test_labls.shape)
  corrects=[0,0,0]
  totals=[0,0,0]
  print(len(test_labls))
  for i in range(len(test_labls)):
    cl=[0,0,0]
    for j in range(5):
      cl[find_class(trees[j],test_y[i])]+=1
    # print(cl)
    freq=0
    id=-1
    for j in range(3):
      if cl[j]>freq:
        freq=cl[j]
        id=j

    if test_labls[i]==id:
      corrects[test_labls[i]]+=1
    totals[test_labls[i]]+=1
  accs=[]
  for i in range(3):
    accs.append(corrects[i]/totals[i])
    print(corrects[i]/totals[i])
  acci=sum(corrects)/sum(totals)
  if(acci>max_acc[0]):
    max_acc=[acci,accs]
  print(sum(corrects)/sum(totals))
print(max_acc)









18623 18623 18623 18623 18623
(784, 1)
[997, 6715, 5507] [4956, 0, 448]
[294, 780, 4583] [5659, 5935, 1372]
[2832, 1345, 3485] [3121, 5370, 2470]
[2351, 3322, 4421] [3602, 3393, 1534]
[2724, 2524, 1733] [3229, 4191, 4222]
[2131, 1937, 2260] [3822, 4778, 3695]
[2805, 4250, 3551] [3148, 2465, 2404]
[5056, 4443, 4493] [897, 2272, 1462]
[3994, 4915, 3584] [1959, 1800, 2371]
[2698, 3506, 2989] [3255, 3209, 2966]
[27, 6664, 1703] [970, 51, 3804]
[101, 780, 4188] [896, 5935, 1319]
[407, 1345, 3355] [590, 5370, 2152]
[591, 3322, 3992] [406, 3393, 1515]
[509, 2524, 1636] [488, 4191, 3871]
[196, 1937, 2045] [801, 4778, 3462]
[143, 4250, 3190] [854, 2465, 2317]
[922, 4443, 4131] [75, 2272, 1376]
[673, 4915, 3300] [324, 1800, 2207]
[308, 3506, 2658] [689, 3209, 2849]
(784, 1)
[4929, 0, 482] [1021, 6696, 5495]
[304, 623, 4613] [5646, 6073, 1364]
[2892, 1620, 3467] [3058, 5076, 2510]
[3775, 3473, 1853] [2175, 3223, 4124]
[3494, 4615, 4492] [2456, 2081, 1485]
[2169, 1908, 2237] [3781, 4788, 3740]
[30